In [312]:
import numpy as np
import numba
import random
import math

def csr_spmv_single_thread(rowptr, colidx, val, x):
    """CSR format based SpMV (y=Ax)"""
    
    num_row = len(rowptr) - 1
    y = np.zeros(num_row, dtype=np.float64)
    
    for i in range(num_row):
        col_start = rowptr[i]
        col_end = rowptr[i + 1]
        for j in range(col_start, col_end):
            y[i] += val[j] * x[colidx[j]]
    
    return y

@numba.jit(nopython=True, parallel=True, cache=True, nogil=True)
def csr_spmv_multi_thread(rowptr, colidx, val, x):
    """CSR format based SpMV (y=Ax)"""
    
    num_row = len(rowptr) - 1
    y = np.zeros(num_row, dtype=np.float64)
    
    for i in numba.prange(num_row):
        col_start = rowptr[i]
        col_end = rowptr[i + 1]
        for j in range(col_start, col_end):
            y[i] += val[j] * x[colidx[j]]
    
    return y

def ellpack_spmv_single_thread(N, b, val, J, x):
    """ELLPACK format based SpMV (y=Ax)"""
    
    y = np.zeros(N, dtype=np.float64)
    for i in range(N):
        for k in range(b):
            if val[k + b * i] != 0:
                y[i] += val[k + b * i] * x[J[k + b * i]]
    
    return y

@numba.jit(nopython=True, parallel=True, cache=True, nogil=True)
def ellpack_spmv_multi_thread(N, b, val, J, x):
    """ELLPACK format based SpMV (y=Ax)"""
    
    y = np.zeros(N, dtype=np.float64)
    for i in numba.prange(N):
        for k in range(b):
            if val[k + b * i] != 0:
                y[i] += val[k + b * i] * x[J[k + b * i]]
    
    return y

def sliced_ellpack_spmv_single_thread(N, slice_ptr, colidx, val, x, slice_height):
    """Sliced ELLPACK format based SpMV (y=Ax)"""
    
    y = np.zeros(N, dtype=np.float64)
    slice_count = int(N / slice_height)
    for s in range(slice_count):
        row_idx = s * slice_height
        for r in range(row_idx, row_idx + slice_height):
            for i in range(slice_ptr[s] + r - row_idx, slice_ptr[s + 1], slice_height):
                y[r] += x[colidx[i]] * val[i]
    
    return y

@numba.jit(nopython=True, parallel=True, cache=True, nogil=True)
def sliced_ellpack_spmv_multi_thread(N, slice_ptr, colidx, val, x, slice_height):
    """Sliced ELLPACK format based SpMV (y=Ax)"""
    
    y = np.zeros(N, dtype=np.float64)
    slice_count = int(N / slice_height)
    for s in numba.prange(slice_count):
        row_idx = s * slice_height
        for r in range(row_idx, row_idx + slice_height):
            for i in range(slice_ptr[s] + r - row_idx, slice_ptr[s + 1], slice_height):
                y[r] += x[colidx[i]] * val[i]
    
    return y

def random_spmatrix(n_row, n_col):
    """Output a random value sparse matrix"""
    
    sp_matrix = []
    nnz_count = 0
    for i in range(n_row):
        row_data = []
        for j in range(n_col):
            r_val = random.randint(0, 15)
            if r_val <= 12:
                row_data.append(0)
            else:
                nnz_count += 1
                row_data.append(r_val)
        sp_matrix.append(row_data)
    
    return sp_matrix, nnz_count

def spmatrix_to_CSR(sp_matrix):
    """Convert sparse matrix to CSR format"""
    
    n_row = len(sp_matrix)
    n_col = len(sp_matrix[0])
    
    rowptr = []
    colidx = []
    val = []
    nnz_count = 0
    
    for i in range(n_row):
        rowptr.append(nnz_count)
        for j in range(n_col):
            if sp_matrix[i][j] != 0:
                nnz_count += 1
                colidx.append(j)
                val.append(sp_matrix[i][j])
    rowptr.append(nnz_count)
    
    return np.array(rowptr), np.array(colidx), np.array(val, dtype=np.float64)

def CSR_to_SELLPACK(rowptr, colidx, val, slice_height):
    """Convert CSR format to Sliced ELLPACK format"""
    
    N = len(rowptr) - 1 # number of rows
    slice_number = math.ceil(N / slice_height) # how many slices
    nnz_count = 0
    
    ell_colidx = []
    ell_sliceptr = []
    ell_val = []
    
    for i in range(slice_number):
        max_nnz = 0
        for s in range(slice_height):
            col_count = rowptr[i * slice_height + s + 1] - rowptr[i * slice_height + s]
            max_nnz = max(max_nnz, col_count)
        
        ell_sliceptr.append(nnz_count)
        for j in range(max_nnz):
            for k in range(slice_height):
                idx = i * slice_height + k # row index
                now_ptr = rowptr[idx] # start index of this row
                next_ptr = rowptr[idx + 1] # start index of next row
                nnz_count += 1 # count non-zero number
                if now_ptr + j < next_ptr:
                    ell_colidx.append(colidx[now_ptr + j])
                    ell_val.append(val[now_ptr + j])
                else:
                    ell_colidx.append(-1) # -1 means invalid
                    ell_val.append(0) # padded zero
    ell_sliceptr.append(nnz_count)
    
    return np.array(ell_colidx), np.array(ell_sliceptr), np.array(ell_val, dtype=np.float64)

In [ ]:
# set number of rows, columns and slice height
n_row = 8000
n_col = 20000
# generate a sparse matrix fill with random value
sp_matrix, nnz_count = random_spmatrix(n_row, n_col)
nnz_per = nnz_count / (n_row * n_col)
print(str(nnz_count) + " non-zero elements in this sparse matrix (" + str(nnz_per) + "%).")

In [335]:
slice_height = 2
# convert sparse matrix to CSR format
csr_rowptr, csr_colidx, csr_val = spmatrix_to_CSR(sp_matrix)
# convert CSR to Sliced ELLPACK
ell_colidx, ell_sliceptr, ell_val = CSR_to_SELLPACK(csr_rowptr, csr_colidx, csr_val, slice_height)
# generate x array
x = np.ones(n_col, dtype=np.float64)
x *= 12.3

In [217]:
%time csr_y = csr_spmv_single_thread(csr_rowptr, csr_colidx, csr_val, x)

Wall time: 20.6 s


In [359]:
%timeit csr_y = csr_spmv_multi_thread(csr_rowptr, csr_colidx, csr_val, x)

14.1 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [232]:
%time ellpack_y = sliced_ellpack_spmv_single_thread(n_row, ell_sliceptr, ell_colidx, ell_val, x, slice_height)

Wall time: 20.7 s


In [358]:
%timeit ellpack_y = sliced_ellpack_spmv_multi_thread(n_row, ell_sliceptr, ell_colidx, ell_val, x, slice_height)

13.4 ms ± 456 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [334]:
sum(csr_y - ellpack_y) # check the result

0.0